In [1]:
import pandas as pd
import numpy as np
import spacy

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
"""
Pré processa os dados os dados do arquivo separando em dois dicionarios de mensagens;
Um chaveado pela data de envio das mensagens;
Um chaveado pelo autor das mensagens.
"""


with open('_chat.txt', 'r') as f:
    data = []
    lines = [line.strip() for line in f.readlines()[6:]]

    for line in lines:
        if line:
            try:
                date = line.split('[')[1].split()[0]
            except:
                continue
            try:
                user = line.split(']')[1].split(':')[0].strip()
            except:
                continue
            
            try:
                message = line.split(':')[3].strip()
            except:
                continue
            data.append((date, user, message))

name_dict = {}
date_dict = {}
for log in data:
    # por nome
    if log[1] in name_dict.keys():
        name_dict[log[1]].append(log[2])
    else:
        name_dict[log[1]] = [log[2]]

    # por data
    if log[0] in date_dict.keys():
        date_dict[log[0]].append(log[2])
    else:
        date_dict[log[0]] = [log[2]]


In [3]:
vectorizer = spacy.load('pt')

In [10]:
"""
Vetoriza os textos de dois membros.
Separa a mesma quantidade em cada vetor
"""
bruno = [vectorizer(text).vector for text in name_dict['Beelzebruno'] if 'sticker' not in text and 'image' not in text]
kamal = [vectorizer(text).vector for text in name_dict['Kamal Curi']  if 'sticker' not in text  and 'image' not in text]

bruno = bruno[:470]
kamal = kamal[:470]

print(len(bruno), len(kamal))

470 470


In [17]:
classes = {
    0: 'Bruno',
    1: 'Kamal',
}

by = [0 for _ in range(470)]
ky = [1 for _ in range(470)]

ys = by + ky
xs = np.array(bruno + kamal)


# Distribuição de classes entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    xs,
    ys
)

In [21]:
# Inicializa um modelo de K-vizinhos mais próximos
# model = KNeighborsClassifier(leaf_size=25, p=1)
model = LogisticRegression(max_iter=1000)

model.fit(list(X_train), y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
# Avalia o modelo
y_pred = model.predict([X_test[0]])
print("Test set score: {:.2f}".format(np.mean(y_pred == y_test)))

Test set score: 0.51


In [40]:
texts = [
    'to vazando',  # bruno
    'Vazando donde',  # kamal
    'só assim mesmo pra dar risada',  # bruno
    'Daquele jeito',  # Kamal
    'o que uma coisa tem haver com a outra?',  # Bruno
    'Jesus Cristo',  # Kamal
    'eu nao tenho uma vida amorosa amigo',  # Bruno
    'hauahauahauahu é foda meu guri',  # Kamal

    'Oh shit',
    'Quem discordar, tá errado.',
    'HAUAHAUAHAUAAJ',
    'eahuhueahuhueahuehua',
    'Tu tá jogando Skyrim agora?'
]

for text in texts:
    text_vector = vectorizer(text).vector
    print(classes[model.predict([text_vector])[0]], text)


Bruno to vazando
Kamal Vazando donde
Bruno só assim mesmo pra dar risada
Kamal Daquele jeito
Bruno o que uma coisa tem haver com a outra?
Kamal Jesus Cristo
Bruno eu nao tenho uma vida amorosa amigo
Kamal hauahauahauahu é foda meu guri
Bruno Oh shit
Bruno Quem discordar, tá errado.
Kamal HAUAHAUAHAUAAJ
Bruno eahuhueahuhueahuehua
Kamal Tu tá jogando Skyrim agora?


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
nn = MLPClassifier(
    hidden_layer_sizes=(250, 250),
    activation='relu',
    solver='adam',
    alpha=0.0001,
    batch_size='auto',
    learning_rate='constant',
    learning_rate_init=0.01,
    power_t=0.5,
    max_iter=1000,
    shuffle=True,
    random_state=None,
    tol=0.001,
    verbose=False,
    warm_start=False,
    momentum=0.9,
    nesterovs_momentum=True,
    early_stopping=False,
    validation_fraction=0.1,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-08,
    n_iter_no_change=10,
    max_fun=15000
)
nn.fit(X_train, y_train)

pred = nn.predict([X_test[0]])
print(f"test set score: {np.mean(pred == y_test):.2f}")
# print(metrics.precision_score(y_test, pred, average='weighted'))